In [1]:
# Importar la librería y las dependencias (todas las funciones nuevas están aquí)
import pandas as pd
import numpy as np
import cufflinks as cf
cf.go_offline() 

import libreria as lb

# Asumimos que X_limpio (atributos A1-A64) y Y (class) están disponibles después del Bloque 1
# X_limpio debe estar limpio de nulos.

In [2]:
#IMPORTAMOS EL DATA SET DE ACUERDO A LA DOCUMENTACIÓN EN https://github.com/uci-ml-repo/ucimlrepo
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
polish_companies_bankruptcy = fetch_ucirepo(id=365) 
  
# data (as pandas dataframes) 
X = polish_companies_bankruptcy.data.features 
y = polish_companies_bankruptcy.data.targets 
  
# metadata 
print(polish_companies_bankruptcy.metadata) 
  
# variable information 
print(polish_companies_bankruptcy.variables) 

{'uci_id': 365, 'name': 'Polish Companies Bankruptcy', 'repository_url': 'https://archive.ics.uci.edu/dataset/365/polish+companies+bankruptcy+data', 'data_url': 'https://archive.ics.uci.edu/static/public/365/data.csv', 'abstract': 'The dataset is about bankruptcy prediction of Polish companies.The bankrupt companies were analyzed in the period 2000-2012, while the still operating companies were evaluated from 2007 to 2013.', 'area': 'Business', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 10503, 'num_features': 65, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 2016, 'last_updated': 'Sat Mar 09 2024', 'dataset_doi': '10.24432/C5F600', 'creators': ['Sebastian Tomczak'], 'intro_paper': {'ID': 417, 'type': 'NATIVE', 'title': 'Ensemble boosted trees with synthetic features generation in application to bankruptcy predictio

Explicación: El Análisis Exploratorio de Datos (EDA) es el primer paso para entender las características principales y los problemas del conjunto de datos. La limpieza es crucial, ya que los datos de baja calidad pueden afectar directamente el rendimiento de los modelos (GIGO: Garbage in -> Garbage out). En este caso, trataremos los valores faltantes (nulos) y exploraremos las distribuciones para identificar outliers.

In [ ]:
print("--- 2.1. Análisis de Valores Faltantes ---")

# 1. Revisar la completitud inicial [5]
print("Porcentaje de nulos antes de la imputación:")
print(completitud_datos(X))

# 2. Imputación de nulos
# Todas las variables A1-A64 son de tipo float64 (continuas) [33].
X_limpio = limpiar_nulos_imputer(X, 
                                 continuous_cols=columnas_financieras, 
                                 discrete_cols=[])

print("\nPorcentaje de nulos después de la imputación:")
print(completitud_datos(X_limpio))

print("\n--- 2.2. Análisis Descriptivo y Outliers (EDA Visual)")

In [ ]:
print("\n--- 2.2. Análisis Descriptivo y Outliers (EDA Visual) ---")

# 3. Visualizar distribuciones para encontrar outliers y sesgos [28]
# Tomamos una muestra de columnas para no saturar la salida
columnas_muestra_eda = ['A1', 'A5', 'A9', 'A10']
generar_histograma_cf(X_limpio, columnas_muestra_eda)

In [ ]:
# 4. Uso de Boxplots para detección visual de Outliers
generar_boxplot_cf(X_limpio, columnas_muestra_eda)


In [ ]:
# 5. Tratamiento de un Outlier (Ejemplo para A5, que tiene alta Std Dev [36])
# El tratamiento de outliers es importante ya que pueden tener una fuerte influencia en los modelos estadísticos [37].
X_filtrado_a5 = tratar_outliers_iqr(X_limpio, 'A5', factor=1.5)
print(f"\nTamaño original (después de imputar): {len(X_limpio)}")
print(f"Tamaño después de filtrar outliers en A5: {len(X_filtrado_a5)}")

# Usaremos X_limpio para los siguientes pasos, ya que PCA y VarClusHi a menudo prefieren la imputación al descarte total.

[Bloque 3: Multicolinealidad y Reducción de Dimensionalidad]
Explicación: Debido a la alta dimensionalidad (64 variables) y la fuerte correlación detectada entre pares de variables (algunas cercanas a 1.0000), es esencial aplicar técnicas de reducción. Esto mejora la interpretación, reduce la redundancia y evita la maldición de la dimensionalidad.

In [ ]:
#3.1. Detección de Multicolinealidad con VarClusHi
print("\n--- 3.1. Detección de Multicolinealidad con VarClusHi ---")

# 1. Escalar los datos antes de VarClusHi (aunque VarClusHi no siempre lo requiere, es buena práctica)
# Usaremos Standard Scaler [12].
X_escalado = escalar_datos(X_limpio, method='standard')

In [ ]:
# 2. Ejecutar VarClusHi
# VarClusHi agrupa variables altamente correlacionadas [20].
resultados_vc = analisis_varclushi(X_escalado)

print("\nResultados de VarClusHi (Top 10):")
print(resultados_vc.head(10))

# **Justificación de selección de representante:**
# Las variables que tienen el 'RS_Ratio' más bajo dentro de su clúster son 
# las mejores representantes, ya que implican una mayor correlación con 
# el componente principal del clúster y mayor diferencia con otros clústeres [14, 23].

In [ ]:
3.2. Reducción de Dimensiones con PCA
print("\n--- 3.2. Reducción con PCA ---")

# 1. Aplicar PCA para retener el 90% de la varianza total [16, 17]
# PCA encuentra las direcciones (componentes) que contienen la máxima varianza [14].
Xp_90 = analisis_pca(X_escalado, n_components=0.90)

print(f"\nNúmero de componentes para el 90% de varianza: {Xp_90.shape[19]}")
print(Xp_90.head())

In [ ]:
# 2. Aplicar PCA para obtener solo 3 componentes (para visualización 3D)
Xp_3 = analisis_pca(X_escalado, n_components=3) 

[Bloque 4: Visualización de Componentes PCA]
Explicación: La visualización es clave para transformar datos en conocimiento. Al reducir el espacio de 64 dimensiones a 2 o 3 componentes, podemos graficar la estructura interna de los datos y observar posibles agrupaciones (clústeres) o outliers.

In [ ]:
print("\n--- 4.1. Visualización de PCA (2D) ---")
# Visualización 2D de los dos primeros componentes
visualizar_pca_componentes(Xp_3.iloc[:, :2], dim=2) 

print("\n--- 4.2. Visualización de PCA (3D) ---")
# Visualización 3D (requiere PC1, PC2, PC3) [44]
visualizar_pca_componentes(Xp_3, dim=3)

Bloque 5: Selección de Variables Supervisada (SelectKBest)
Explicación: La Selección de Variables Predictivas es un método supervisado (depende de la variable objetivo, Y) para combatir la Maldición de la Dimensionalidad y mejorar la interpretabilidad del modelo. SelectKBest utiliza pruebas estadísticas (como F-classif para clasificación) para asignar una puntuación a cada variable, priorizando aquellas que tienen la asociación más fuerte con la variable objetivo.

In [ ]:
print("--- 5.1. Aplicación de SelectKBest ---")

# La variable objetivo Y (class) es binaria (0/1), lo que requiere F_classif o Chi2 [4].
# Usaremos F_classif para datos continuos.
# X_escalado proviene del Bloque 3 (data limpia y estandarizada).
Y = df_polish['class'] # Variable objetivo binaria (0/1)

In [ ]:
# Aplicar SelectKBest para seleccionar las 7 mejores variables por defecto
# Nota: La función SelectKBest requiere que X contenga valores no negativos para chi2 [5].
# Usaremos f_classif que funciona bien con datos escalados (StandardScaler).
X_kbest_7, scores_kbest = seleccionar_kbest(X_escalado, Y, k=7, score_func='f_classif')

In [ ]:
print(f"\nLas 7 mejores variables seleccionadas son:")
print(X_kbest_7.columns.tolist())

# Visualizar el ranking de todas las variables (Poder Predictivo)
# scores_kbest.set_index('Variable')['Puntuación'].iplot(kind='bar', title='Puntuación KBest por Variable')

# Justificación: Solo se conservan las 7 variables que estadísticamente son más predictivas de Y.

Bloque 6: Transformación Entrópica (WoE y IV)
Explicación: El Peso de la Evidencia (WoE) y el Valor de Información (IV) son herramientas cruciales en la modelización de clasificación binaria (como el riesgo crediticio o la bancarrota). El WoE transforma las categorías de una variable en una nueva escala numérica basada en el logaritmo de las probabilidades de Eventos (bancarrota) vs No Eventos (no bancarrota). El IV resultante cuantifica el poder predictivo de esa variable.
6.1. Conceptos Teóricos
• Peso de la Evidencia (WoE): Indica si una categoría tiene una mayor o menor proporción de eventos (ej. bancarrota) que la población total. Si WoE > 0, el grupo tiene más No Eventos ('Buenos'); si WoE < 0, tiene más Eventos ('Malos').
• Valor de Información (IV): Mide el poder predictivo total. Un IV alto (0.3 a 0.5) es considerado un predictor fuerte; valores bajos (< 0.02) son inútiles.
• Requisito: Las variables continuas (A1-A64) deben ser discretizadas (binning) antes de aplicar WoE/IV.


In [ ]:
print("\n--- 6.2. Transformación Entrópica (IV y WoE) ---")

# Usaremos la variable A1 (activos / total de pasivos) como ejemplo.
FEATURE_NAME = 'A1' 

# 1. Discretización de la variable continua (binning) [7]
# Creamos 5 bins basados en cuantiles.
A1_binned = discretizar_variable_para_woe(X_limpio, feature=FEATURE_NAME, n_bins=5)

In [ ]:
# Crear un DataFrame temporal con la variable binned y el target
df_woe = pd.DataFrame({'A1_binned': A1_binned, 'class': Y})

# 2. Calcular WoE e IV
# Calculamos WoE por cada rango de A1 y el IV total para la variable A1
resultados_woe_iv, iv_total = calcular_woe_iv_variable(
    df_woe, 
    feature='A1_binned', 
    target='class'
)

print(f"\nResultados WoE y IV para la variable {FEATURE_NAME} (Discretizada en 5 bins):")
print(resultados_woe_iv)
print(f"\nValor de Información (IV) total para {FEATURE_NAME}: {iv_total:.4f}")

In [ ]:
# Evaluación del Poder Predictivo:
if iv_total < 0.02:
    print("IV sugiere que la variable es impredecible / inútil [9].")
elif 0.1 <= iv_total <= 0.3:
    print("IV sugiere que la variable es un predictor medio [9].")

El proceso que hemos completado es similar a la labor de un cartógrafo que intenta representar un planeta complejo (el dataset de 64 dimensiones) en un mapa plano (2D o 3D). Primero, el cartógrafo debe limpiar el territorio (imputar nulos y tratar outliers). Luego, utiliza PCA para encontrar los ejes de rotación más significativos del planeta (los Componentes Principales), permitiéndonos comprimir la inmensa cantidad de información original en un dibujo simple pero representativo, facilitando la toma de decisiones y la identificación de regiones clave (patrones o grupos de empresas)